# Working on a Baseline

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import json
import time
import string
# from tqdm import *
import datetime
import math

In [15]:
#############################################################################
## READ IN THE CSVs WITH ALL THE DATA AND REDO THE DATETYPE TRANSFORMATION ## 
#############################################################################
traindf = pd.read_csv('/Users/william/DropBox/CS 109/traindf.csv', index_col=0);
traindf.date = pd.to_datetime(traindf.date)
traindf['name'] = traindf['name'].astype(str)
traindf['name'] = map(lambda x: x.upper(), traindf['name'])
# traindf.loc[traindf['win']=='NaN'] = 0.0
# traindf.loc[traindf['place']=='NaN'] = 0.0
# traindf.loc[traindf['show']=='NaN'] = 0.0
# traindf['win'].fillna(0, inplace=True)

testdf = pd.read_csv('tempdata/testdf.csv', index_col=0);
testdf.date = pd.to_datetime(testdf.date)
testdf['name'] = testdf['name'].astype(str)
testdf['name'] = map(lambda x: x.upper(), testdf['name'])
testdf[['win', 'place', 'show']].fillna(0, inplace=True)
testdf = testdf[testdf.date >= datetime.datetime(1999,1,1)]

In [3]:
traindf = traindf[traindf.name != 'NAN']
traindf.reset_index().drop('index', axis=1, inplace=True);

In [16]:
testdf.head(100)

,post,name,win,place,show,fin_pos,Odds,date,track,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
11,6,CHARISMATIC,NaN,NaN,NaN,1,8.4,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
12,5,MENIFEE,NaN,NaN,NaN,2,2.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
13,4,BADGE,NaN,NaN,NaN,3,58.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
14,11,STEPHEN GOT EVEN,NaN,NaN,NaN,4,11.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
15,8,PATIENCE GAME,NaN,NaN,NaN,5,15.6,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
16,6,ADONIS,NaN,NaN,NaN,6,19.8,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
17,3,CAT THIEF,NaN,NaN,NaN,7,5.2,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
18,2,KIMBERLITE PIPE,NaN,NaN,NaN,8,17.4,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
19,12,VALHOL,NaN,NaN,NaN,9,61.7,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
20,13,VICAR,NaN,NaN,NaN,10,24.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0


In [12]:
#traindf["win_indicator"] = (traindf["fin_pos"] == 1)*1
#testdf["win_indicator"] = (testdf["fin_pos"] == 1)*1

notcols_train = ["racenum", "date", "place", "show"]
temp_train = (set(traindf.columns) - set(notcols))

notcols_test = ["place", "show", ]
print temp
#xtrain = traindf[cols''
#ytrain = traindf["win"]

set(['minhumidity', 'maxhumidity', 'meanwindspdm', 'meanpressurem', 'meandewptm', 'win', 'snow', 'hail', 'precipm', 'meanvism', 'track', 'snowfallm', 'fin_pos', 'meanwdird', 'fog', 'snowdepthm', 'thunder', 'rain', 'post', 'meantempm', 'name'])


In [ ]:
def find_races(horses_name, traindf=traindf):
    return traindf[traindf['name']==horses_name]

def find_avg_payouts(df, race_date=datetime.date.today()):
    out = {}
    df = df[df.date < race_date]
    out['win'] = np.mean(df.win[df.win != 0.0])
    out['place'] = np.mean(df.place[df.place != 0.0])
    out['show'] = np.mean(df.show[df.show != 0.0])
    return out

find_avg_payouts(find_races('SMARTY JONES', traindf=testdf), race_date=datetime.datetime(1998, 12, 12))

The payouts are already standardized to $2 bets, so we luckily don't have to deal with that problem

In [ ]:
unique_test_dates = testdf['date'].unique()

def find_horses(date):
    return testdf['name'].loc[testdf['date'] == date]

def find_track(date):
    return testdf['track'].loc[testdf['date'] == date][0]

#find_horses(datetime.datetime(1998, 5, 15))
#type(testdf['date'][0])

We run a simple regression (aka Average) to get a dict where the keys are a tuple `(date, track, horse)` and the values are themselves dictionaries with keys `win`, `place`, `show` and values as the corresponding averages leading into raceday (excluding information we couldn't have on raceday). 

In [ ]:
avg_payouts = {}
for i in testdf['date'].unique():
    race = testdf.loc[testdf.date == i]
    for j in range(len(race)):
        date = race.date.iloc[j]
        track = race.track.iloc[j]
        horse = race.name.iloc[j]
        avg_payouts[(date, track, horse)] = find_avg_payouts(
            find_races(race['name'].iloc[j], traindf=traindf), race_date=date)

In [ ]:
## this function takes a payout and returns the first odds digit, assuming a comparison to 1, i.e. x-1 
## of that horse to win when betting occured assuming original bet of $2 and assumed take by race track of 15% 
def payoff_to_odds(payoff, bet_amount=2.0, take = .15):
    return round(((payoff/(1-take) - bet_amount)/bet_amount),4)

def odds_to_percent(odds): 
    return (float(str(odds)[2]))/(float(str(odds)[0])+float((str(odds)[2])))

def normalize_odds(odds): 
    x = odds.split("-")
    if len(x) > 1: 
        return float(x[0])/float(x[1])
    else: 
        return float(x[0])
    
def make_favorite(string): 
    if "favorite" in string: 
        return True 
    else:
        return False 

In [ ]:
def baseline(indict):
    profit = []
    cost = []
    count = []
    count2 = []
    for key in indict.keys():
        count2.append(1)
        date = key[0]
        track = key[1]
        horse = key[2]
        
        exp_win = indict[key]['win']
        exp_place = indict[key]['place']
        exp_show = indict[key]['show']
        
        # get the morning line odds for that day to compare
        odds = testdf.loc[(testdf.name == horse) & (testdf.track == track) & (testdf.date == date)]['Odds'].iloc[0]

        if odds > payoff_to_odds(exp_win):
            cost.append(2)
            
            if math.isnan(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0]):
                count.append(1)
                pass
            
            else:
                profit.append(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0])
        else:
            count.append(1)
            pass
        
    return ['baseline', no_bets_placed, race, year, profit_wins, profit_places, profit_shows, total_profit, cost]
            
baseline(avg_payouts)

Dataframe of race, year, profit/loss on win baseline, profit/loss on place baseline, profit/loss on show baseline

In [ ]:
def make_bets_baseline(indict, unique_test_dates=unique_test_dates): 
    outdict = {}
    outdict['method'] = []
    outdict['no_bets_placed'] = []
    outdict['race'] = []
    outdict['year'] = []
    outdict['profit_wins'] = []
    outdict['profit_places'] = []
    outdict['profit_shows'] = []
    outdict['total_profit'] = []
    outdict['cost'] = []
    for unique_date in unique_test_dates:
        newdict = {}
        for key in indict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]
            
            
            if date == unique_date:
                newdict[key] = indict[key]
                  
        # now we have newdict, subsetting based on race (by unique date)
        
        spread_tup = []
        for key in newdict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]

            exp_win = newdict[key]['win']
            exp_place = newdict[key]['place']
            exp_show = newdict[key]['show']

            # get the morning line odds for that day to compare
            odds = testdf.loc[(testdf.name == horse) & 
                              (testdf.track == track) & 
                              (testdf.date == date)]['Odds'].iloc[0]
            
            spread = odds - payoff_to_odds(exp_win)
            spread_tup.append((horse, spread))
            
        sorted_list = sorted(spread_tup,key=lambda x: x[1], reverse=True)
        
        print sorted_list[0:3]
            
#         outdict['method'] = []
#         outdict['no_bets_placed'] = []
#         outdict['race'] = []
#         outdict['year'] = []
#         outdict['profit_wins'] = []
#         outdict['profit_places'] = []
#         outdict['profit_shows'] = []
#         outdict['total_profit'] = []
#         outdict['cost'] = []

make_bets_baseline(avg_payouts)